## Imports

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

## Paths

In [ ]:
TRAIN = '../Data/df_train.csv'
TEST = '../Data/df_test.csv'

## Functions

In [ ]:
z_score = lambda x : (x-np.mean(x,axis=0))/ np.std(x,axis=0) #code from exercise 5

## Variables and df loading

In [ ]:
col = 'tab10' #colour theme

df = pd.read_csv(TRAIN) #training dataframe

attributes = list(df.columns)[:-1] #creates list of column names for later

X = df[attributes].copy() #attributes
y = df['type'].copy() #true values

y_types = {1:'wind_float', 2:'wind_non', 3:'wind_veh', 5:'container', 6:'tableware', 7:'headlamp'}
y_list = y.unique() #'y' values

X_std = z_score(X) #standardizes X as later we see they are on different scales

for i in y_list: #this changes the 'type' to a descriptive word rather than a number
    #This is more human readable for plotting later - 'y' is still using the numbers
    df['type'][df['type'] == i] = y_types[i]

In [ ]:
def Gini(df, y, threshold):
    '''Takes a threshold and calc the Gini impurity of each split of the given df and class'''
    df = pd.concat([df,y], axis=1,)
    df.columns=['Att', 'class']
    # Split the Input data into side a and b
    df_a = df[df['Att'] >= threshold] #check what the accepted standard for the equals bit is
    df_b = df[df['Att'] < threshold]
    # calc and return gini impurity for each side
   
    gini_a = 0
    gini_b = 0

    for class_ in y_list:
        
        # Add total of each class
        class_tot_a = 0
        class_tot_b = 0
        for i in range(len(df_a)):
            # Add the total in a of this class
            if df_a['class'].iloc[i] == class_:
                class_tot_a +=1
        for i in range(len(df_b)):
            # Add the total in b of this class
            if df_b['class'].iloc[i] == class_:
                class_tot_b +=1 
        #print(class_tot_a)
        
        # side a
        p_a = class_tot_a/len(df_a) # proportion of this class out of total points on side a (k/N)
        # Probability of class
        left = p_a *(1-p_a) 
        
        # b
        p_b = class_tot_b/len(df_b) # proportion of this class out of total points on side b
        # Probability of class
        right = p_b *(1-p_b)
    
        gini_a += left
        gini_b += right
        
    return ((len(df_a)/len(df))*gini_a) + ((len(df_b)/len(df))*gini_b)

    
print(Gini(df['Ba'], y, 0.4))

In [ ]:
def Gini_2(df, y, threshold):#, class_):
    df = pd.concat([df,y], axis=1,)
    df.columns=['Att', 'class']
    '''Takes a threshold and calc the Gini impurity of each split of the given df and class'''
    
    # Split the Input data into side a and b
    df_a = df[df['Att'] >= threshold]
    df_b = df[df['Att'] < threshold]
    
    left_class_count = np.zeros((8), dtype=int)
    right_class_count = np.zeros((8), dtype=int)
    for i in range(len(df_a)):
        left_class_count[df_a['class'].iloc[i]]+=1
    for i in range(len(df_b)):
        right_class_count[df_b['class'].iloc[i]]+=1        
    left_class_count = np.delete(left_class_count, [0,4])
    right_class_count = np.delete(right_class_count, [0,4])
    
    print(left_class_count, right_class_count)
    
    left_proportion = [left_class_count[i]/np.sum(left_class_count) for i in range(len(left_class_count))]
    right_proportion = [right_class_count[i]/np.sum(right_class_count) for i in range(len(right_class_count))]
    print(left_proportion, right_proportion)
    
    left_gini = 0
    for i in range(len(left_proportion)):
        left_gini += left_proportion[i]*(1-left_proportion[i])
    
    right_gini = 0
    for i in range(len(right_proportion)):
        right_gini += right_proportion[i]*(1-right_proportion[i])    
    print(left_gini, right_gini)
    
    Gini = (left_gini * sum(left_class_count)/len(df)) + (right_gini * sum(right_class_count)/len(df))
    print(Gini)

    return()

    
print(Gini_2(df['Ba'], y, 0.4))

In [ ]:
df = pd.concat([df['Ba'],y], axis=1,)
df.columns=['Att', 'class']

class_count = np.zeros((8), dtype=int)
for i in range(len(df)):
    class_count[df['class'].iloc[i]]+=1        
class_count = np.delete(class_count, [0,4])
print(class_count)
    
proportion = [class_count[i]/np.sum(class_count) for i in range(len(class_count))]
print(proportion)
    
gini = 0
for i in range(len(proportion)):
    gini += proportion[i]*(1-proportion[i])
    
#Gini = (left_gini * sum(left_class_count)/len(df)) + (right_gini * sum(right_class_count)/len(df))
print(gini)

In [ ]:
def leaf_hunter(df=df, class_list=y_list):
    '''takes df and class returns highest gini gain, best class, best threshold, and 
    best attribute'''
    #So many loops! 
    best_gini_gain = 0
    best_class = ''
    best_threshold = ''
    best_att = ''
    
    for cl in y_list:
        '''loop through classes'''
        for a in attributes:
            tmp = df.sort_values(by=a, axis=0).copy()
                #print(a)
                #loop through attributes
            for threshold in range(len(df)-1):
                '''loop through potential thresholds'''
                if tmp[a].iloc[threshold] == tmp[a].iloc[threshold+1]:
                    pass
                else:
                    # Threshold is halfway in between two datapoints
                    thresh = ((tmp[a].iloc[threshold] + tmp[a].iloc[threshold+1])/2)
                    #print((tmp[a].iloc[threshold], tmp[a].iloc[threshold+1]), thresh)       
                    g_gain = Gini(df[a], y, thresh)#, cl)
                    if g_gain > best_gini_gain:
                        best_gini_gain = g_gain
                        best_class = cl
                        best_threshold = thresh
                        best_att = a
    return best_gini_gain, best_class, best_threshold, best_att
            
best_gini_gain, best_class, best_threshold, best_att = leaf_hunter()

print(best_gini_gain, best_class, best_threshold, best_att)

In [ ]:
plt.scatter([i for i in range(149)], df[best_att], c=y, cmap=col)
plt.hlines(best_threshold, 0, 149);

In [ ]:
clf = DecisionTreeClassifier(max_depth=1)
model = clf.fit(X, y)

In [ ]:
fig = plt.figure(figsize=(10,7))
_ = tree.plot_tree(model, filled= True)

In [ ]:
X.head()

In [ ]:
for a in attributes:
    print(a, len(X[X[a] <= 0.4]))

In [ ]:
plt.scatter([i for i in range(149)], df['Ba'], c=y, cmap=col)
plt.hlines(0.4, 0, 149);